<a href="https://colab.research.google.com/github/chasubeen/ASCEND/blob/%EC%A7%80%ED%98%84/1_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EA%B0%80%EA%B3%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 코랩에서 실행하는 경우 drive 마운트 필요

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

import os
import glob
import gc
import pyarrow.parquet as pq
import time

# **1. 데이터 확인**

- Google Colab에 로컬 런타임을 연결하여 사용
  - 파일 경로는 로컬 컴퓨터 기준

In [ ]:
base_path = './data'

## 해당 경로에서 .parquet으로 끝나는 파일 목록 가져오기
parquet_files = glob.glob(os.path.join(base_path, '*.parquet'))

# 파일들을 정렬하여 순차적으로 처리
parquet_files.sort()

## 월별로 파일 읽어오기
for file_path in parquet_files:
    # 파일명 추출
    file_name = os.path.basename(file_path)

    # 연도, 월 정보 추출
    year = file_name.split('-')[2]
    month = file_name.split('-')[3]

    # 데이터 로딩
    df = pd.read_parquet(file_path)

    # 결측치 확인
    print("=== {}-{} ===".format(year, month))
    print(df.isna().sum())

    # 데이터프레임을 메모리에서 삭제
    del df

    # 가비지 컬렉션 수동 실행
    gc.collect()

=== 2023-01.parquet ===
id                0
price             0
qty               0
quote_qty         0
time              0
is_buyer_maker    0
dtype: int64
=== 2023-02.parquet ===
id                0
price             0
qty               0
quote_qty         0
time              0
is_buyer_maker    0
dtype: int64
=== 2023-03.parquet ===
id                0
price             0
qty               0
quote_qty         0
time              0
is_buyer_maker    0
dtype: int64
=== 2023-04.parquet ===
id                0
price             0
qty               0
quote_qty         0
time              0
is_buyer_maker    0
dtype: int64
=== 2023-05.parquet ===
id                0
price             0
qty               0
quote_qty         0
time              0
is_buyer_maker    0
dtype: int64
=== 2023-06.parquet ===
id                0
price             0
qty               0
quote_qty         0
time              0
is_buyer_maker    0
dtype: int64
=== 2023-07.parquet ===
id                0
price         

- 결측치는 존재하지 않는다.

# **2. OHLCV 데이터로 가공하기**

In [ ]:
### OHLCV 데이터 가공 함수

def convert_tick_to_ohlcv(data):
    """
    주어진 Binance 틱 데이터를 1시간 간격의 OHLCV (Open, High, Low, Close, Volume) 데이터로 변환
    :param data: DataFrame with Tick data
    :return: DataFrame with the Open(시가), High(고가), Low(저가), Close(종가), Volume(거래량) values
    """

    ## datetime 형으로 변환
    data['time'] = pd.to_datetime(data['time'], unit='ms')

    ## 시간대별로 재정렬
    data = data.sort_values(by = 'time').reset_index(drop = True)

    ## ohlcv 데이터로 가공
    # 1시간 단위로 resampling
    ohlcv = data.resample('1H', on='time').agg({
        'price': ['first', 'max', 'min', 'last'],  # 시가, 고가, 저가, 종가
        'qty': 'sum',  # 거래량
    })

    # 컬럼 이름 변경
    ohlcv.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    # time열 추가
    ohlcv['Time'] = ohlcv.index

    # 열 순서 조정
    ohlcv = ohlcv[['Time', 'Open', 'High', 'Low', 'Close', 'Volume']]

    return ohlcv

In [ ]:
## 주어진 경로
base_path = './data'

## 해당 경로에서 .parquet으로 끝나는 파일 목록 가져오기
parquet_files = glob.glob(os.path.join(base_path, '*.parquet'))

# 파일들을 정렬하여 순차적으로 처리
parquet_files.sort()

## 결과를 저장할 빈 리스트 생성
final_dfs = []

## 월별로 파일을 읽어오고 전처리 수행
for file_path in parquet_files:
    # 연도, 월 정보 추출
    year = file_name.split('-')[2]
    month = file_name.split('-')[3]

    # 각 월별 데이터를 읽어와 ohlcv 데이터로 변환
    df = pd.read_parquet(file_path)
    processed_df = convert_tick_to_ohlcv(df)

    # 최종 결과 리스트에 추가
    final_dfs.append(processed_df)

    # 데이터프레임을 메모리에서 삭제
    del processed_df

    # 가비지 컬렉션 수동 실행
    gc.collect()

    print("{}-{} 전처리 완료!".format(year, month))

## 리스트에 저장된 모든 DataFrame을 합치기
final_result_df = pd.concat(final_dfs, ignore_index = True)

2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!
2024-01.parquet 전처리 완료!


In [ ]:
# 시간에 따라 재정렬
final_result_df = final_result_df.sort_values(by='Time').reset_index(drop=True)

# index를 Time으로 설정
final_result_df.set_index('Time', inplace=True)

In [ ]:
# 최종 결과 확인
final_result_df

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-01 00:00:00,16537.5,16540.9,16504.0,16527.0,5381.399
2023-01-01 01:00:00,16527.1,16554.3,16524.1,16550.4,3210.826
2023-01-01 02:00:00,16550.5,16557.1,16534.8,16542.4,2399.668
2023-01-01 03:00:00,16542.5,16542.5,16515.0,16529.3,3214.480
2023-01-01 04:00:00,16529.2,16530.4,16508.8,16517.8,3150.954
...,...,...,...,...,...
2024-01-27 19:00:00,41785.0,41977.0,41775.1,41896.4,7765.565
2024-01-27 20:00:00,41896.5,42070.0,41896.4,42049.6,5408.243
2024-01-27 21:00:00,42049.6,42165.6,42001.8,42137.8,5859.277


In [ ]:
final_result_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9408 entries, 2023-01-01 00:00:00 to 2024-01-27 23:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    9384 non-null   float64
 1   High    9384 non-null   float64
 2   Low     9384 non-null   float64
 3   Close   9384 non-null   float64
 4   Volume  9408 non-null   float64
dtypes: float64(5)
memory usage: 441.0 KB


- 중간에 결측치가 있는 것 같음 -> 적절한 처리 필요

In [ ]:
# 결측인 시간대 확인

final_result_df[final_result_df.isnull().any(axis=1)]

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-23 21:00:00,NaN,NaN,NaN,NaN,0.0
2023-01-23 22:00:00,NaN,NaN,NaN,NaN,0.0
2023-01-23 23:00:00,NaN,NaN,NaN,NaN,0.0
2023-01-24 21:00:00,NaN,NaN,NaN,NaN,0.0
2023-01-24 22:00:00,NaN,NaN,NaN,NaN,0.0
2023-01-24 23:00:00,NaN,NaN,NaN,NaN,0.0
2023-01-25 16:00:00,NaN,NaN,NaN,NaN,0.0
2023-01-25 17:00:00,NaN,NaN,NaN,NaN,0.0
2023-01-25 18:00:00,NaN,NaN,NaN,NaN,0.0


# **3. 결측치 핸들링**

In [ ]:
# Open, High, Low, Close -> 보간
final_result_df[['Open', 'High', 'Low', 'Close']] = final_result_df[['Open', 'High', 'Low', 'Close']].interpolate(method = 'time')

# Volume -> 0일 수가 없기에 0을 결측치로 간주, 해당 월의 평균값으로 처리
final_result_df.loc[final_result_df['Volume'] == 0, 'Volume'] = None
final_result_df['Volume'] = final_result_df.groupby(final_result_df.index.month)['Volume'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
final_result_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9408 entries, 2023-01-01 00:00:00 to 2024-01-27 23:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    9408 non-null   float64
 1   High    9408 non-null   float64
 2   Low     9408 non-null   float64
 3   Close   9408 non-null   float64
 4   Volume  9408 non-null   float64
dtypes: float64(5)
memory usage: 699.0 KB


- 결측치가 제대로 처리되었다.

In [ ]:
### 파일로 저장해두기

final_result_df.to_csv('./차수빈/final/OHLCV.csv', encoding = 'utf-8')